In [ ]:
# STEP 1: Install Required Libraries
!pip install roboflow gradio pillow_heif==0.11.1 --quiet

# STEP 2: Download Dataset using Roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="ljOV2xnWYPwLbinaMbIv")
project = rf.workspace("fakharsworkspace").project("ripescan-2")
version = project.version(1)
dataset = version.download("folder")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ripescan-2-1 in folder:: 100%|██████████| 1347/1347 [00:00<00:00, 9040.50it/s]


In [ ]:
# Import libraries
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import gradio as gr
from tensorflow.keras.preprocessing import image

# Set paths
train_path = "/content/ripescan-2-1/train"
test_path = "/content/ripescan-2-1/test"
validation_path = "/content/ripescan-2-1/valid"

# Load sample image
def load_sample_image(path):
    for category in os.listdir(path):
        category_path = os.path.join(path, category)
        image_path = os.path.join(category_path, os.listdir(category_path)[0])
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))
        return img
fig = px.imshow(load_sample_image(train_path))
fig.update_layout(title="Sample Image from Training Set")
fig.show()

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path, target_size=(224, 224),
                                                    batch_size=32, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_path, target_size=(224, 224),
                                                        batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path, target_size=(224, 224),
                                                  batch_size=32, class_mode='categorical', shuffle=False)

# Build VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(train_generator, epochs=3, validation_data=validation_generator)

# Fine-tune model
base_model.trainable = True
for layer in base_model.layers[:15]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_fine = model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Evaluate model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy*100:.2f}%")

# Classification report
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print("Classification Report")
print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))

# Confusion matrix
cm = confusion_matrix(test_generator.classes, y_pred)
cm_fig = px.imshow(cm, labels=dict(x="Predicted", y="True", color="Count"),
                   x=list(test_generator.class_indices.keys()),
                   y=list(test_generator.class_indices.keys()), text_auto=True)
cm_fig.update_layout(title="Confusion Matrix")
cm_fig.show()

# Accuracy plot
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines', name='Train Accuracy'))
fig.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines', name='Validation Accuracy'))
fig.update_layout(title='Model Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy')
fig.show()

# Loss plot
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation Loss'))
fig.update_layout(title='Model Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

# Fine-tuning Accuracy plot
fig_fine = go.Figure()
fig_fine.add_trace(go.Scatter(y=history_fine.history['accuracy'], mode='lines', name='Fine-tune Train Accuracy'))
fig_fine.add_trace(go.Scatter(y=history_fine.history['val_accuracy'], mode='lines', name='Fine-tune Validation Accuracy'))
fig_fine.update_layout(title='Fine-tuning Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy')
fig_fine.show()

# Fine-tuning Loss plot
fig_fine_loss = go.Figure()
fig_fine_loss.add_trace(go.Scatter(y=history_fine.history['loss'], mode='lines', name='Fine-tune Train Loss'))
fig_fine_loss.add_trace(go.Scatter(y=history_fine.history['val_loss'], mode='lines', name='Fine-tune Validation Loss'))
fig_fine_loss.update_layout(title='Fine-tuning Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig_fine_loss.show()

# Save model
model.save('/kaggle/working/fruit_veg_classifier.h5')

# Gradio Interface
class_names = list(test_generator.class_indices.keys())

def classify_fruit_veg(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    predictions = model.predict(img_array)[0]
    top_index = np.argmax(predictions)
    confidence = predictions[top_index]
    predicted_label = class_names[top_index]
    return f"Prediction: {predicted_label} ({confidence * 100:.2f}%)"

interface = gr.Interface(
    fn=classify_fruit_veg,
    inputs=gr.Image(type="pil", label="Upload Fruit or Vegetable Image"),
    outputs="text",
    title="🍎🥦 Smart Sorting Classifier",
    description="Upload an image of a fruit or vegetable(EX:Banana,Mango,Tomato). The model will predict the class using transfer learning (VGG16)."
)

interface.launch(share=True)


: 